In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['items.csv', 'sample_submission.csv', 'test.csv', 'sales_train.csv', 'item_categories.csv', 'shops.csv']


In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [3]:
sales = pd.read_csv('../input/sales_train.csv')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
items = pd.read_csv('../input/items.csv')
test = pd.read_csv('../input/test.csv')
item_categories = pd.read_csv('../input/item_categories.csv')
shops = pd.read_csv('../input/shops.csv')

sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')

- prediction are need to be made based on monthly data

In [5]:
# create grid of shop and items and add sum() of their item_cnt_day (0 for NaN)
index_cols = ['date_block_num', 'shop_id', 'item_id']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[[block_num], cur_shops, cur_items])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#####################
#get aggregated values for (shop_id, item_id, month)
#gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target': 'sum'}})
#fix column names
#gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]

# alternative way to sum item_cnt_day, without warning from pandas for using dict in agg()
gb = sales.groupby(index_cols,as_index=False).item_cnt_day.sum()
gb = gb.rename(columns={'item_cnt_day': 'item_cnt_month'})
#####################

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [9]:
all_data.head(10)

,date_block_num,shop_id,item_id,item_cnt_month
139255,0,0,19,0.0
141495,0,0,27,0.0
144968,0,0,28,0.0
142661,0,0,29,0.0
138947,0,0,32,6.0
138948,0,0,33,3.0
138949,0,0,34,0.0
139247,0,0,35,1.0
142672,0,0,40,0.0
142065,0,0,41,0.0


- Create target mean encoding feature with CV loop regularization using KFold

In [21]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

all_data['item_target_enc'] = np.NaN

for rest, curr in kf.split(all_data):
    # divide the data using folds
    #rest_fold, cur_fold = all_data[rest], all_data[curr]
    
    # use rest_fold to calculate mean target
    #all_data.iloc[curr]['item_target_enc'] = all_data.iloc[rest].groupby('item_id')['item_cnt_month'].transform('mean')
    # 8e-16
    # should be all_data.iloc[curr, all_data.columns.get_loc('item_target_enc')]
    
    #item_id_target_mean = all_data.iloc[rest].groupby('item_id').item_cnt_month.mean()
    #all_data.ix[curr,'item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
    #.ix is depreciated
    # 0.41650796922
    
    item_id_target_mean = all_data.iloc[rest].groupby('item_id').item_cnt_month.mean()
    all_data.loc[all_data.index[curr],'item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
    # 0.4164590712798811
    
    #all_data.ix[curr, 'item_target_enc'] = all_data.iloc[rest].groupby('item_id')['item_cnt_month'].transform('mean')
    # 0.0801429988734
    
# Fill NaNs
all_data['item_target_enc'].fillna(0.33427, inplace=True)

encoded_feature = all_data['item_target_enc'].values

# compute correlation
np.corrcoef(all_data['item_cnt_month'].values, encoded_feature)[0][1]

0.41645904630880237

In [20]:
all_data.item_cnt_month.mean()

0.3342730567123426

In [22]:
all_data.to_csv('all_data.csv', index=False)

In [17]:
pd.read_csv('all_data.csv')

,date_block_num,shop_id,item_id,item_cnt_month,item_target_enc
0,0,0,19,0.0,0.334300
1,0,0,27,0.0,0.048523
2,0,0,28,0.0,0.142424
3,0,0,29,0.0,0.030303
4,0,0,32,6.0,0.894020
5,0,0,33,3.0,0.485238
6,0,0,34,0.0,0.107018
7,0,0,35,1.0,0.020408
8,0,0,40,0.0,0.054717
9,0,0,41,0.0,0.038136
